In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import heapq
from collections import defaultdict

In [7]:
data = pd.read_csv("./datasets/credit_card.csv").fillna(0)

Normalizing data

In [8]:
datamax = data.iloc[:,1:].max(axis=0)
data1 = data.iloc[:,1:].divide(datamax,axis=1)
data = pd.concat([data.iloc[:,0], data1],axis=1).fillna(0)

Distance Function

In [9]:
from functools import partial
L2 = partial(np.linalg.norm,ord=2)

Pre-calculating pair-wise distances of all points (useful in greedy algo as well as calculating optimum)

In [10]:
n_points = data.shape[0]
dist = [[0 for i in range(n_points)] for j in range(n_points)]
points = [np.array(data.iloc[i].iloc[1:]) for i in range(data.shape[0])]

In [11]:
for i in tqdm(range(n_points-1)):
    for j in range(i+1,n_points):
        d = L2(points[i]-points[j])
        dist[i][j]=dist[j][i]=d

In [12]:
np.save("dist.npy",np.array(dist))

In [13]:
dist = np.load("./dist.npy")

### Greedy Algorithm - (2 Approximation)

In [14]:
def farthest(clusters):
    argmax, maxd = -1,-1
    for i in range(len(clusters)):
        d2c = dist[clusters[i]][i]
        if d2c > maxd:
            maxd = d2c
            argmax = i
    return argmax, maxd

In [15]:
def greedy(points, k, first=-1):
    n = len(points)
    pind = list(range(n))
    if first==-1:
        first = np.random.choice(pind)
    k-=1
    clusters = [first]*n
    centers = [first]
    newc,dk = farthest(clusters) 
    while k:
        centers.append(newc)
        for i in range(n):
            if dist[i][clusters[i]]>dist[i][newc]:
                clusters[i]=newc
        k-=1
        newc,dk = farthest(clusters)
    return centers, clusters, dk

In [16]:
first = np.random.choice(range(len(points)))

In [21]:
from IPython.display import display

In [32]:
ans = []
ks = [2,4,8,20]
for k in ks:
    cnt, clus, dk = greedy(points,k,first=first)
    ans.append([k,dk])
ans = pd.DataFrame(ans)
ans.columns = ["K", "Score"]
display(ans)

,K,Score
0,2,2.199641
1,4,1.920288
2,8,1.546847
3,20,1.179808


### Finding Optimal k-centers

In [23]:
def opt2(points):
    n = len(points)
    optc = []
    optd2 = 10**9+7
    for c1 in tqdm(range(n-1)):
        for c2 in range(c1+1,n):
            d2 = np.max(np.minimum(distnp[:,c1],distnp[:,c2]))
            if d2 < optd2:
                optd2=d2;optc=[c1,c2]
    c1,c2 = optc
    clusters = [c1]*n
    for i in range(n):
        if distnp[i][c2]<distnp[i][c1]:
            clusters[i]=c2
    return optc, clusters, optd2

In [24]:
def opt3(points):
    n = len(points)
    optc = []
    clusters = []
    optd3 = 10**9+7
    for c1 in tqdm(range(n-2)):
        for c2 in range(c1+1,n-1):
            for c3 in range(c2+1,n):
                d3 = np.max(np.minimum(np.minimum(distnp[:,c1],distnp[:,c2]),distnp[:,c3]))
                if d3 < optd3:
                    optc, optd3 = [c1,c2,c3], d3
    c1,c2,c3 = optc
    clusters = [c1]*n
    for i in range(n):
        if distnp[i][c2]<distnp[i][c1]:
            clusters[i]=c2
        if distnp[i][c3]<distnp[i][clusters[i]]:
            clusters[i]=c3
    return optc, clusters, optd3

Taking only a random portion (1%) of the dataset for finding optimal.

In [25]:
indices = np.random.choice(range(len(points)),len(points)//100,replace=False)
points_1p = [points[i] for i in indices]
distnp = np.array([[dist[i][j] for i in indices] for j in indices])

In [33]:
ans = []

In [34]:
_,__, optd2 = opt2(points_1p)
_,__, greedyd2 = greedy(points_1p,2,first=first)
ans.append([2, optd2, greedyd2, greedyd2/optd2])

In [35]:
_,__, optd3 = opt3(points_1p)
_,__, greedyd3 = greedy(points_1p,3,first=first)
ans.append([3, optd3, greedyd3, greedyd3/optd3])

In [36]:
ans = pd.DataFrame(ans)
ans.columns = ["K", "Optimal Score", "Greedy Score", "Approximation Ratio"]
display(ans)

,K,Optimal Score,Greedy Score,Approximation Ratio
0,2,1.179486,1.501574,1.273075
1,3,1.087222,1.438984,1.323542


### Improving Performance

I tried finding the approximation hardness of the k-centers problem. I found that it is actually $2$, i.e. if there is a $(2-\epsilon)$ approximation for k-centers, then P=NP. 

They mentioned that it can be proved by a reduction from Dominating Set, which I tried and I created a simple instance of k-centers given an instance of Dominating Set. 

I just put weights on the edges as $1$, then distance is defined as the shortest distance between the two points in the graph. So, if there is a dominating set of size $k$, then there exists a k-center clustering with $\Delta_k = 1$, else it is greater than $1$. Our approximation algorithm must give answer $(2-\epsilon)\times 1$, but that is not possible as the edge weights are in multiples of $1$. So, if the dominating set exists, then the approximation algorithm returns a solution with $\Delta_k = 1$, else something greater than 1. By this we created a solution for Dominating Set. 

Hence, the approximation hardness is $2$.

On this dataset, I propose the following heuristic algorithm:

1) Find greedy solution.<br>
2) In each cluster, re-assign the center such that it still covers all the points in that cluster, but the radius is minimized(this is like brute forcing a k=1 solution on a smaller set of points).<br>
3) Now, as the centers are changed, assign all the points to their nearest centers.<br>
4) Recalculate the final score/cost of the clustering.

As we can see, this algorithm tries to optimize the already existing greedy algorithm. If it is possible to optimize, it will, else it is equal to the greedy solution. Therefore, My_algo $\le$ Greedy.

The time complexity of this algorithm is time complexity of greedy + $\sum_{i=1}^{k} p_i^2$ + $\mathcal{O}(nk)$(for re-assigning centers), where p_i denotes the no. of points in $i^{th}$ cluster.

In worst case, it will be $\mathcal{O}(kn^2)$

In [30]:
def heuristic(points, k, first=-1):
    n = len(points)
    cnts, clus, dk = greedy(points, k, first)
    cluss = defaultdict(list)
    for i in range(n):
        cluss[clus[i]].append(i)
    new_cnt = []
    for clu in cluss:
        best = 10**9+7
        besti = -1
        for i in cluss[clu]:
            rad = 0
            for j in cluss[clu]:
                rad = max(rad, dist[i][j])
            if rad<best:
                best=rad
                besti=i
        new_cnt.append(besti)
    new_clus = [new_cnt[0]]*n
    for i in new_cnt[1:]:
        for j in range(n):
            if dist[j][i]<dist[j][new_clus[j]]:
                new_clus[j]=i
    _, dk = farthest(new_clus)
    return new_cnt, new_clus, dk

In [45]:
ans = [20,1,1]
for first in tqdm(range(10)): # Just running on few different start points and fetching best
    _, __, greedy_dk = greedy(points, k=20, first=first)
    _, __, dk = heuristic(points,k=20, first=first)
    if greedy_dk-dk > ans[1]-ans[2]:
        ans[1],ans[2] = greedy_dk,dk
ans = pd.DataFrame([ans],columns = ["K","Greedy Score","My Algo Score"])
display(ans)

,K,Greedy Score,My Algo Score
0,20,1.166002,1.002642


As we can see, my algo has an improvement over the greedy algorithm. Also, the time taken is around 10-15 sec, for given start point.